# Protein Ligand Complex

You can run this notebook in your browser: [![Open On Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sef43/openmm_workshop/blob/main/exercise_1/exercise_1_part2.ipynb)


In this notebook we will demonstrate two ways of setting up a simulation of a protein-ligand complex in OpenMM.

- **Method 1**

  Using the OpenMM package [openmmforcefields](https://github.com/openmm/openmmforcefields) and an external package called Open Force Field toolkit ([openff-toolkit](https://github.com/openforcefield/openff-toolkit)).

  This covers the following steps:
    - Loading in the ligand with `openff-toolkit`.
    - Parameterising the ligand force-field with `openmmforcefields`.
    - Combining the topologies.
    - Solvating and simulating.

  *Note this notebook is based on the [openff-toolkit's example](https://github.com/openforcefield/openff-toolkit/blob/stable/examples/toolkit_showcase/toolkit_showcase.ipynb) . We would like to give credit to the Open ForceField Authors.*

- **Method 2**

  Using a third party tool ([BioSimSpace](https://biosimspace.openbiosim.org/)) to produce OpenMM compatible input files.

  This covers the following steps:
    - Using a BSS workflow to take the protein+ligand files and produce Amber format input files.
    - Using Amber input files with OpenMM.


## Table of Contents
- [Method 1: OpenFF-toolkit](#method1)
  - [Extra conda packages](#packages1)
  - [System](#system)
  - [Load in the molecules](#load)
  - [Create the forcefield](#createff)
  - [Combine and solvate](#combine)
  - [Simulate](#simulate1)
- [Method 2: BioSimSpace](#method2)
  - [Extra conda packages](#packages2)
  - [BSS Workflow](#bssworkflow)
  - [Run with OpenMM](#run)
- [Extra exercises](#extraex)
- [Solutions](#solutions)

## Method 1: OpenFF-toolkit 
<a id="method1"></a>

### Extra packages
<a id="packages1"></a>

We will need to install the additional python packages:

 - openmmforcefields
   - github: https://github.com/openmm/openmmforcefields
   - conda-forge: https://anaconda.org/conda-forge/openmmforcefields
 - openff-toolkit
   - github: https://github.com/openforcefield/openff-toolkit
   - conda-forge: https://anaconda.org/conda-forge/openff-toolkit

Both of these will be installed if you install openmmforcefields from conda-forge.

Note for apple silicon you may need to create a x86 conda environment see [here](https://github.com/openforcefield/openff-toolkit/blob/main/FAQ.md#im-having-troubles-installing-the-openff-toolkit-on-my-apple-silicon-mac).

If you run into problems we recommend you create a fresh conda environment and install openmmforcefields firsts. It is easier for conda to solve dependency issues in fresh environments.

In [ ]:
# Execute this cell to install mamba in the Colab environment

if 'google.colab' in str(get_ipython()):
  print('Running on colab')
  !pip install -q condacolab
  import condacolab
  condacolab.install_mambaforge()
else:
  print('Not running on colab.')
  print('Make sure you create and activate a new conda environment!')

In [ ]:
!mamba install -y -c conda-forge openmmforcefields

### Imports

We need to be careful with the imports here because OpenMM and OpenFF have some objects with the same names. For this reason we no longer use the wildcard imports and adopt a more typical Python programming approach.

In [ ]:
# OpenMM imports
import openmm.app as app
import openmm as mm
import openmm.unit as unit
from sys import stdout

from openmmforcefields.generators import SMIRNOFFTemplateGenerator


# OpenFF-toolkit imports
from openff.toolkit import Molecule
from openff.toolkit import Topology as offTopology
from openff.units.openmm import to_openmm as offquantity_to_openmm


### System
<a id="system"></a>

Our example system will be a complex of a benzene ligand and a lysozyme protein. The lysozyme is an antimicrobial protein that has been extensively studied by MD simulations. We can download the files from the github repo. We will also download a second ligand (o-xylene) for use in an exercise.

In [ ]:
# get the files
!wget https://raw.githubusercontent.com/sef43/openmm_workshop/main/section_1/benzene.sdf
!wget https://raw.githubusercontent.com/sef43/openmm_workshop/main/section_1/o-xylene.sdf
!wget https://raw.githubusercontent.com/sef43/openmm_workshop/main/section_1/lysozyme.pdb

The benzene-lysozyme complex in shown in the figure below.

![benzene-lysozyme](benzene_lysozyme.png)
**Figure.** Benzene-lysozyme complex.

Note that the files we are using have already been cleaned up (see [PDBFixer](https://github.com/openmm/pdbfixer) for more info). Additionally, the ligand is aligned with the protein and in an appropriate binding site. This is something you would need to do with a docking program before using OpenMM.


### Load in the molecules
<a id="load"></a>

The protein is a PDB file so we can load it as before. The benzene molecule is in SDF file format for which OpenMM does not have loaders, therefore we will we use OpenFF-toolkit to load it in.


In [ ]:
protein_path = "lysozyme.pdb"
ligand_path = "benzene.sdf"

# Load a molecule from a SDF file
ligand = Molecule.from_file(ligand_path)

# Load in the protein from a PDB file
protein_pdb = app.PDBFile(protein_path)

### Creating the ForceField
<a id="createff"></a>

We now need to define the forcefield to use. For the protein we can use the standard forcefields already available in OpenMM. For the benzene molecule we will need to generate a forcefield template for it.

We can do this using the residue template generators for small molecules from the [openmmforcefields](https://github.com/openmm/openmmforcefields) package. There is a choice between the [Amber GAFF small molecule force field](http://ambermd.org/antechamber/gaff.html) or the [Open Force Field Initiative force fields](https://github.com/openforcefield/openff-forcefields).

For this example we will use [OpenFF SMIRNOFF](https://docs.openforcefield.org/projects/toolkit/en/stable/users/smirnoff.html).


In [ ]:
# Create the SMIRNOFF template generator with the default installed force field
smirnoff = SMIRNOFFTemplateGenerator(molecules=ligand)

# we can check which version of the force field is being used
print(smirnoff.smirnoff_filename)

# Create an OpenMM ForceField object with AMBER ff14SB and TIP3P
ff = app.ForceField('amber/protein.ff14SB.xml', 'amber/tip3p_standard.xml')

# add in the SMIRNOFF template generator
ff.registerTemplateGenerator(smirnoff.generator)

### Combine topologies and solvate
<a id="combine"></a>

We can convert from the OpenFF format topology to an OpenMM format topology and then use the OpenMM Modeller to combine the ligand and protein into a single topology. Once combined we can solvate as before.

In [ ]:
# make an OpenMM Modeller object with the protein
modeller = app.Modeller(protein_pdb.topology, protein_pdb.positions)

# make an OpenFF Topology of the ligand
ligand_off_topology = offTopology.from_molecules(molecules=[ligand])

# convert it to an OpenMM Topology
ligand_omm_topology = ligand_off_topology.to_openmm()

# get the positions of the ligand
ligand_positions = offquantity_to_openmm(ligand.conformers[0])

# add the ligand to the Modeller
modeller.add(ligand_omm_topology, ligand_positions)

# solvate
modeller.addSolvent(ff, padding=1.0*unit.nanometer, ionicStrength=0.15*unit.molar)


### Simulate
<a id="simulate1"></a>

We can now simulate in the NVT ensemble as before.

**Exercise 1.** You will need to set the initial positions

In [ ]:
system = ff.createSystem(modeller.topology, nonbondedMethod=app.PME, constraints=app.HBonds)
integrator = mm.LangevinMiddleIntegrator(300*unit.kelvin, 1/unit.picosecond, 0.002*unit.picoseconds)
simulation = app.Simulation(modeller.topology, system, integrator)


# set the positions
FIXME


print("Minimizing energy...")
simulation.minimizeEnergy(maxIterations=100)

simulation.context.setVelocitiesToTemperature(300*unit.kelvin)

simulation.reporters.append(app.PDBReporter('traj.pdb', 100))

simulation.reporters.append(app.StateDataReporter(stdout, 100, step=True,
        potentialEnergy=True, temperature=True, speed=True))

print("Running simulation...")
simulation.step(1000)

## Method 2
<a id="method2"></a>

In this method we use a [BioSimSpace](https://biosimspace.openbiosim.org/) (BSS) workflow to produce [Amber](https://ambermd.org/) format input files than we can read into OpenMM.

### Extra packages
<a id="packages2"></a>

We will need to install [biosimspace from the openbiosim channel](https://anaconda.org/openbiosim/biosimspace)

The conda command below tells conda to look in the `openbiosim` and `conda-forge` channels for biosimspace and the optional, but required for this workflow, dependencies gromacs and ambertools.

In [ ]:
!mamba install -y  -c conda-forge -c openbiosim biosimspace  gromacs ambertools

### Get the BSS workflow
<a id="bssworkflow"></a>

We will use a BSS workflow (also called a Node) which takes in the ligand SDF file along with protein PDB file and combines them into a protein-ligand complex solvated in a water box. A BSS node is a python script that can be run as a command line program. We will use the script in this workshop repo which is based on an [example script from BioSimSpace](https://github.com/michellab/BioSimSpace/blob/6a36648e1f2e95ee6de35b2e6c9ac32f201c2bc8/nodes/playground/BSSPrepNode.ipynb). For more information please look at the [BioSimSpace documentation](https://biosimspace.openbiosim.org/).

In [ ]:
# get the BSS workflow
!wget https://raw.githubusercontent.com/sef43/openmm_workshop/main/section_1/BSSPrepNode.py

### Run the workflow

If you run the script without any command line arguments it will print out help info.

In [ ]:
!python BSSPrepNode.py

We can then run it specifying the ligand and protein files with command line arguments.

In [ ]:
if 'google.colab' in str(get_ipython()):
  os.environ["AMBERHOME"]="/usr/local/"
else:
  print('You might need to set AMBERHOME env variable')
import os

!python BSSPrepNode.py --ligand benzene.sdf --protein lysozyme.pdb

It will produce the files:
 - `bound.prm7` - the Amber topology file for the protein-ligand complex.
 - `bound.rst7` - the Amber coordinate file for the protein-ligand complex.

it also produces `free.prm7` and `free.rst7` which are the input files for just the ligand solvated in a water box. These would be used in thermodynamic cycle calculations to compute binding energies.

### Run with OpenMM
<a id="run"></a>

OpenMM has the capability to load [Amber format files](http://docs.openmm.org/latest/userguide/application/02_running_sims.html#using-amber-files). `AmberPrmtopFile` can load in the topology file and `AmberInpcrdFile` can load in the coordinates. The rest of the setup is very similar to using PDB files. Note that we do not need to define a forcefield as this is included in the Amber topology file so we call `prmtop.createSystem` instead of `ff.createSystem`.

**Exercise 2.** You will need to specify the integrator.

In [ ]:
import openmm.app as app
import openmm as mm
import openmm.unit as unit
from sys import stdout

prmtop = app.AmberPrmtopFile('bound.prm7')
inpcrd = app.AmberInpcrdFile('bound.rst7')
system = prmtop.createSystem(nonbondedMethod=app.PME, nonbondedCutoff=1*unit.nanometer,
        constraints=app.HBonds)

# specify the integrator
integrator = FIXME

simulation = app.Simulation(prmtop.topology, system, integrator)
simulation.context.setPositions(inpcrd.positions)

print("Minimizing energy...")
simulation.minimizeEnergy(maxIterations=100)

simulation.context.setVelocitiesToTemperature(300*unit.kelvin)
simulation.reporters.append(app.PDBReporter('bss_traj.pdb', 100))
simulation.reporters.append(app.StateDataReporter(stdout, 100, step=True,
        potentialEnergy=True, temperature=True))

print("Running simulation...")
simulation.step(1000)

## Extra exercises
<a id="extraex"></a>

- **Exercise 3.** Run both the setups again but use the o-xylene ligand.
- **Exercise 4.** Take a look at the BSSPrepNode.py in a text editor. Can you modify it to only output the bound state?
- **Exercise 5.** Make a modified version of Method 1 to run a simulation of just the ligand in a waterbox.
- **Exercise 6.** Modify both methods to run in the NPT ensemble.

## Solutions
<a id="solutions"></a>